<font size=3> Applied Machine Learning - Dissecting the Spotify valence.
    <br> Panagiotis Kaliosis - p3180061 </font>

<font size =3><br> We begin by importing the libraries that we are going to need. </font>

In [12]:
import pandas as pd

import glob
import re
from datetime import datetime

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
import seaborn as sns
import scipy.stats.stats as stats
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline

In [13]:
#pip install ggplot

<font size=3><br> Loading our dataset...</font>

In [14]:
header = 0
dfs = []
for file in glob.glob('Charts/global/201?/*.csv'):
    dates = re.findall('\d{4}-\d{2}-\d{2}', file.split('/')[-1])
    weekly_chart = pd.read_csv(file, header=header, sep='\t')
    weekly_chart['week_start'] = datetime.strptime(dates[0], '%Y-%m-%d')
    weekly_chart['week_end'] = datetime.strptime(dates[1], '%Y-%m-%d')
    dfs.append(weekly_chart)

all_charts = pd.concat(dfs)
all_charts

,position,song_id,song_name,artist,streams,last_week_position,weeks_on_chart,peak_position,position_status,week_start,week_end
0,1,5aAx2yezTd8zXrkmtKl66Z,Starboy,The Weeknd,25734078,NaN,1,1,new,2016-12-30,2017-01-06
1,2,7BKLCZ1jbUBVqRi2FVlTVw,Closer,The Chainsmokers,23519705,NaN,1,2,new,2016-12-30,2017-01-06
2,3,5knuzwU65gJK7IF5yJsuaW,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,21216399,NaN,1,3,new,2016-12-30,2017-01-06
3,4,4pdPtRcBmOSQDlJ3Fk945m,Let Me Love You,DJ Snake,19852704,NaN,1,4,new,2016-12-30,2017-01-06
4,5,3NdDpSvN911VPGivFlV5d0,I Don’t Wanna Live Forever (Fifty Shades Darke...,ZAYN,18316326,NaN,1,5,new,2016-12-30,2017-01-06
...,...,...,...,...,...,...,...,...,...,...,...
195,196,0fySG6A6qLE8IvDpayb5bM,VIBEZ,DaBaby,5560742,158.0,13,53,-38,2019-12-20,2019-12-27
196,197,7CZyCXKG6d5ALeq41sLzbw,Take What You Want (feat. Ozzy Osbourne & Trav...,Post Malone,5548309,99.0,15,16,-98,2019-12-20,2019-12-27
197,198,5a6pdCHlWS2ekOOQ70QnAr,July,Noah Cyrus,5525805,156.0,7,138,-42,2019-12-20,2019-12-27
198,199,25ZAibhr3bdlMCLmubZDVt,QUE PRETENDES,J Balvin,5502509,177.0,26,15,-22,2019-12-20,2019-12-27


<font size=3><br> Checking the distinct number of songs that can be found in the dataset and dropping the duplicates. </font>

In [15]:
len(all_charts['song_id'].unique())

3021

In [16]:
all_charts = all_charts.drop_duplicates('song_id')

In [17]:
len(all_charts)

3021

<font size=3><br> So the size of the dataset is 3021 unique songs. 
<br> Now it is time to load the tracks' features. We will use spotipy library, creating a Spotify client in order to access the data. </font>

In [18]:
from spotify_config import config

client_credentials_manager = SpotifyClientCredentials(config['client_id'],
                                                      config['client_secret'])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
#sp.trace = False

ModuleNotFoundError: No module named 'spotify_config'

In [ ]:
features = {}
all_track_ids = list(all_charts['song_id'].unique())

<font size=3><br> Loading the tracks' features in 100-sized batches. </font>

In [ ]:
start = 0
num_tracks = 100
while start < len(all_track_ids):
    print(f'getting from {start} to {start+num_tracks}')
    tracks_batch = all_track_ids[start:start+num_tracks]
    features_batch = sp.audio_features(tracks_batch)
    features.update({ track_id : track_features 
                     for track_id, track_features in zip(tracks_batch, features_batch) })
    start += num_tracks

<font size=3><br> Let's see an example of a song's features. </font>

In [ ]:
features['7qiZfU4dY1lWllzX7mPBI3']

<font size=3><br> Turning the dictionary into a DataFrame.
<br> Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_dict.html </font>

In [ ]:
feats = pd.DataFrame.from_dict(features, orient='index')

In [ ]:
feats.head(10)

<font size=3><br> Dropping the unnecessary columns, as they are unable to help us for the goals of this assignment. </font>

In [ ]:
feats=feats.drop(columns=['type', 'id', 'uri', 'track_href', 'analysis_url'])

In [ ]:
feats = feats.reset_index()

<font size=3><br> Planning on joining the "song_id" and "song_name" columns of the all_charts dataframe into the feats dataframe. So I extract these two columns into a separate dataframe. </font>

In [ ]:
song_names = all_charts[['song_id', 'song_name']]

In [ ]:
feats.head(5)

In [ ]:
feats['index']

<font size=3><br> Joining the two dataframes on the "song_id" attribute.
<br> Source: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html </font>

In [ ]:
feats = feats.join(song_names.set_index('song_id'), on='index')

<font size=3><br>Now moving the "song_name" column at the beginning of the dataframe in order to look better. </font>

In [ ]:
move_first = feats.pop('song_name')
feats.insert(0, 'song_name', move_first)
feats.head(5)

<font size=4><br> Q1 - Explore which Features Influence Valence </font>

In [ ]:
feats.columns[11]

<font size=3><br> A common way to measure correlation between two or more variables is the Pearson correlation coefficient.
So, the script below is used to measure the Pearson correlation coefficient between the valence and all of the other track features. You can read more about Pearson's correlation here: https://stackabuse.com/calculating-pearson-correlation-coefficient-in-python-with-numpy/ </font>

In [ ]:
pearson = pd.DataFrame(columns=['feature', 'correlation'])
X = feats.valence
for item in range(2, 12):
    if item != 11:
        Y = feats[feats.columns[item]]
        out = stats.pearsonr(X,Y)
        row = {'feature': feats.columns[item], 'correlation': out[0]}
        pearson = pearson.append(row, ignore_index = True)

pearson

<font size=3><br>As we can see, there is no strong linear correlation with any of the other features. The stronger ones are with energy, danceability and loudness, although they are not that strong. But this was a really simple approach. We are going to be more advanced.</font>

<font size=3><br> The stronger linear correlation is the one between valence and energy. But it still does not look really linear. </font>

In [ ]:
ggplot() + geom_point(aes(x=feats.valence, y=feats.energy))

<font size=3><br> Let's try Spearman's correlation, which is another metric for measuring the correlation between 2 or more variables. The Spearman's correlation is more sensitive to outliers.
<br> More: https://statistics.laerd.com/statistical-guides/spearmans-rank-order-correlation-statistical-guide.php </font>

In [ ]:
spearman = pd.DataFrame(columns=['feature', 'correlation'])
X = feats.valence
for item in range(2, 12):
    if item != 11:
        Y = feats[feats.columns[item]]
        out = stats.spearmanr(X,Y)
        row = {'feature': feats.columns[item], 'correlation': out[0]}
        spearman = spearman.append(row, ignore_index = True)

spearman

<font size=3><br> The Spearman's correlation results are more or less the same. As a first approach, we could say that danceability, energy and loudness are the three features that look more correlated with the valence. </font>

<font size=3><br> Regression </font>

<font size=3><br> We'll start by performing a regression of the valence value on the energy variable. </font>

In [ ]:
mod = smf.ols("valence ~ energy", data=feats)
feats_res = mod.fit()
feats_res.params
feats_res.summary()

<font size=3><br> The R-squared metric is 0.135, which is not really high. Let's plot the data and the regression line. </font>

In [ ]:
ggplot(data=feats) +\
    geom_point(mapping=aes(x='energy', y='valence')) +\
    geom_abline(intercept=feats_res.params['Intercept'], 
                slope=feats_res.params['energy'])

<font size=3><br> Now let's plot the residuals in order to get a better undestanding of the situation. </font>

In [ ]:
ggplot(data=feats) +\
    geom_hline(yintercept=0, color='black', linetype='dotted') +\
    geom_point(mapping=aes(x='energy', y=feats_res.resid))

<font size=3><br> And of course the QQ-plot. There seems to be a correlation between the two variables but it is not that strong. </font>

In [ ]:
ggplot(data=feats_res.resid.to_frame().rename(columns={0: 'resid'})) +\
    geom_qq(mapping=aes(sample='resid')) +\
    geom_qq_line(mapping=aes(sample='resid'))

<font size=3><br> We will now proceed to add the danceability feature on our regression. </font>

In [ ]:
mod_ml = smf.ols("valence ~ energy + danceability", data=feats)
feats_res_ml = mod_ml.fit()
feats_res_ml.summary()

<font size=3><br> The R-squared metric is a bit better now, with a value of 0.205. But still not really satisfactory. </font>

<font size=3><br> And finally, the loudness feature is added into the mix. </font>

In [ ]:
mod_ml = smf.ols("valence ~ energy + danceability + loudness", data=feats)
feats_res_ml = mod_ml.fit()
feats_res_ml.summary()

<font size=3><br> The truth is that not that much changed after the loudness feature has been added to the regression variables. </font>

<font size=3><br> Now let's check what happens if we make a regression involving all of the song's features. </font>

In [ ]:
all_columns = list(feats.columns[2:13])
all_columns.remove('valence')
all_columns_formula = "valence ~ " + '+'.join(all_columns)
all_columns_formula

In [ ]:
mod_ml_all = smf.ols(formula=all_columns_formula, data=feats)
feats_res_ml_all = mod_ml_all.fit()
feats_res_ml_all.summary()

<font size=3><br> Now, let's try the interaction terms and squares, which could possibly help. </font>

In [ ]:
mod_it = smf.ols("valence ~ danceability + np.power(danceability, 2)", data=feats)
feats_res_it = mod_it.fit()
feats_res_it.summary()

<font size=3><br> Looks like the squares dont fit the problem that well.
<br> Now running an ANOVA test. The null hypothesis is that the valence~energy model fits the problem better than the valence~{all of the features} one.
<br> More on the ANOVA test here: https://www.statisticshowto.com/probability-and-statistics/hypothesis-testing/anova/ </font>

In [ ]:
table = sm.stats.anova_lm(feats_res, feats_res_it)
table

<font size=3><br> As we can see the p-value is equal to 1, which proves our null hypothesis. </font>

<font size=3><br>Now let's perform a regression using interaction terms between the three most valence-correlated variables (danceability, energy and loudness). </font>

In [ ]:
mod_inter = smf.ols("valence ~ danceability*energy*loudness", data=feats)
feats_res_inter = mod_inter.fit()
feats_res_inter.summary()

<font size=3><br> And finally an ANOVA test, where the null hypothesis is that the valence~energy model fits the problem better than the valence ~ danceability * energy * loudness problem. </font>

In [ ]:
table = sm.stats.anova_lm(feats_res, feats_res_inter)
table

<font size=3><br> The p-value is essentially equal to 0, which means that our null hypothesis is not true. So, the interaction terms model fits the problem better. This means that the three variables (danceability, energy, loudness) seem to be the most important ones. They play a bigger role on what the valence value will be. </font>

<font size=3><br> Subsets now. </font>

In [ ]:
def process_subset(y, data, feature_set):
    X = data.loc[:, feature_set].values
    X = sm.add_constant(X)
    names = ['intercept']
    names.extend(feature_set)
    model = sm.OLS(y, X)
    model.data.xnames = names
    #print(model.data.xnames)
    regr = model.fit()
    return regr

In [ ]:
import itertools

def get_best_of_k(y, data, k):
    
    best_rsquared = 0
    best_model = None
    for comb in itertools.combinations(data.columns[2:], k):
        regr = process_subset(y, data, comb)
        if regr.rsquared > best_rsquared:
            best_rsquared = regr.rsquared
            best_model = regr

    return best_model

In [ ]:
def best_subset_selection(data, exog):
    best_model = None
    best_models = []
    y = data.loc[:, exog]
    endog = [ x for x in data.columns[2:] if x != exog ]
    X = data.loc[:, endog]

    for i in range(1, len(data.columns[2:12])):
        print(f'Finding the best model for {i} variable{"s" if i > 1 else ""}')
        model = get_best_of_k(y, X, i)
        if not best_model or model.rsquared_adj > best_model.rsquared_adj:
            best_model = model
        print(f'past that {i}')
        print(model.model.data.xnames[1:]) # get the variables minums the intercept
        best_models.append(model)

    print(f'Fitted {2**len(data.columns)} models')
    return best_model, best_models

In [ ]:
best_model, _ = best_subset_selection(feats, 'valence')

In [ ]:
mod_subsets = smf.ols("valence ~ loudness + duration_ms + speechiness + acousticness", data=feats)
feats_res_subsets = mod_subsets.fit()
feats_res_subsets.summary()

<font size=3><br>Overall, I come to the conclusion that the valence variable mostly depends on the danceability, energy and loudness features, which is more or less really reasonable. Every happy song that passes through my mind can be identified as a really danceable, energetic and -most of the times- loud song. Although, I think that valence could also be influenced from the song's lyrics, for example analyzing the lyrics using NLP algorithms and adding the output to the overall evaluation. </font>

<font size=3><br> Q2 - Predict Valence</font>

In [ ]:
move_ = feats.pop('tempo')
feats.insert(11, 'tempo', move_)

In [ ]:
feats.head()

<font size=3><br> KNN - The first non neural network algorithm that we are going to use is the K-Nearest Neighbours algorithm.
<br> We split the dataset into train and test data using the train_test_split sklearn's function. </font>

In [ ]:
from sklearn.model_selection import train_test_split

X = feats.loc[:, 'danceability':'tempo'].values
y = feats['valence'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40)

<font size=3><br>It is time to define the KNeighboursRegressor and then fit the model. </font>

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=6)
neigh.fit(feats.loc[:, 'danceability':'tempo'].values, feats['valence'].values)

<font size=3><br> Let's see a prediction of the model. </font>

In [ ]:
print(neigh.predict([[0.497, 0.694, 8, -5.677, 1, 0.0478, 0.0811, 0.00001, 0.176, 202]]))

<font size=3><br> With the score function, we receive an evaluation of the model. More specifically, we receive the R-squared metric on the given samples. </font>

In [ ]:
neigh.score(X, y) #R squared

<font size=3><br> Now, it is time to find which is the best value for K. We use the script below and evaluate each K based on the Root Mean Squared Error (RMSE) metric.</font>

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from numpy import sqrt

rmse_val = [] #to store rmse values for different k
for K in range(20):
    K = K+1
    model = KNeighborsRegressor(n_neighbors = K)

    model.fit(X_train, y_train)  #fit the model
    pred=model.predict(X_test) #make prediction on test set
    error = sqrt(mean_squared_error(y_test,pred)) #calculate rmse
    rmse_val.append(error) #store rmse values
    print('RMSE value for k = ' , K , 'is:', error)

<font size=3><br> The RMSE is averagely around 0.2. So best K is probaly K=6 or K=7.
<br> Below there is a plot of the RMSE for the different values of K.</font>

In [ ]:
#plotting the rmse values against k values
curve = pd.DataFrame(rmse_val) #elbow curve 
curve.plot()

<font size=3><br> So let's use the train and test data into the model. </font>

In [ ]:
model = KNeighborsRegressor(n_neighbors = 7)

model.fit(X_train, y_train)  #fit the model
pred=model.predict(X_test) #make prediction on test set

<font size=3><br> A new dataframe is created in order to store the knn estimation and the true valence for each song and see them side-by-side. </font>

In [ ]:
valence_estimator_2_df = pd.DataFrame(columns=['knn_estimation', 'true_valence'])
valence_estimator_2_df['true_valence'] = y_test

In [ ]:
valence_estimator_2_df['knn_estimation'] = pred

<font size=3><br> So this is the ouput, and the look of the newly created dataframe. </font>

In [ ]:
valence_estimator_2_df

<font size=3>-----------------------------------------------------------------------------------------</font>

<font size=3><br>Linear Regression - The second non neural network model that we are going to use is the linear regression.
<br> For this purpose, we are once again using the sklearn library. </font>

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)

<font size=3><br> Once again, let's see the evaluation of the model. Firstly, on the train data. </font>

In [ ]:
reg.score(X_train, y_train)

<font size=3><br> The R-squared value for the train data is 0.24 which is not really that good.
<br> Let's now check the coefficients of the regression. </font>

In [ ]:
reg.coef_

<font size=3><br> And of course, the final evaluation, on the test data. </font>

In [ ]:
reg.score(X_test, y_test)

<font size=3>-----------------------------------------------------------------------------------------</font>

<font size=3><br>Decision Tree -  The third and final non neural network method that is going to be used on this assignment is the decision tree. </font>

In [ ]:
feats.columns

In [ ]:
cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness','tempo']

<font size=3><br> For one more time, we preprocess the data and then splitting them apropriately into train and test sets. </font>

In [ ]:
from sklearn.model_selection import train_test_split

X = feats.loc[:, cols].values
y = feats['valence'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40)

<font size=3><br> Below is the intialization of the decision tree. It is stated with a max_depth of 8, min_samples_leaf equal to 0.13 and random state = 3. We fit the train data into the decision tree. </font>

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtree = DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)

dtree.fit(X_train, y_train)

<font size=3><br> Now let's use the decision tree in order to predict the valence of the songs. We are first using the model on the train data. </font>

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

pred_train_tree= dtree.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_tree)))
print(r2_score(y_train, pred_train_tree))

<font size=3><br> As we can see, the RMSE is equal to 0.19 and the R-squared metric equal to 0.20. 
<br> Now let's use the tree on the test data. </font>

In [ ]:
pred_test_tree= dtree.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_tree))) 
print(r2_score(y_test, pred_test_tree))

<font size=3><br>As expected the R-squared value is lower than it was on the train data. 
<br> But there are some ways in order to improve the performance of the decision tree. We are now going to create a random forest.
<br> First we import the RandomForestRegressor and then create the model. Of course we fit the train data into the model.</font>

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#RF model
model_rf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=100)
model_rf.fit(X_train, y_train) 

<font size=3><br> Now let's check the efficiency of the Random Forest. First in the train data and then in the test data. </font>

In [ ]:
pred_train_rf= model_rf.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_rf)))
print(r2_score(y_train, pred_train_rf))

<font size=3><br> Actually, the Random Forest Regressor is performing really well in the train data. </font>

In [ ]:
pred_test_rf = model_rf.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_rf)))
print(r2_score(y_test, pred_test_rf))

<font size=3><br> Moreover, the Random Forest Regressor is performing way better than the simple decision tree on the test data. There is a lower RMSE (=0.16) and much higher R-squared (=0.40). </font>

<font size=3>Neural Network - It is now time to build a neural network model in order to estimate the value of the valence for all these songs.
<br> The Tensorflow and Keras libraries are going to be used. </font>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers

In [ ]:
X_train_df = pd.DataFrame(X_train, columns = cols)

In [ ]:
X_train_df.describe().transpose()

<font size=3><br> Before building the model, we have to preprocess the data. As we can clearly see, there is a big variation between the different features' values. So we are going to normalize the data using the Normalization function that the Keras library is offering. </font>

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers

normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(X_train_df))
with np.printoptions(precision=2):
    print(normalizer.mean)
    print(normalizer.variance)
    print(normalizer.count)

<font size=3><br>Now, it is time to build the model. After the normalization of the data, there will be 5 dense layers. The first layer will have 512 neurons and RELU as activation function. Layers 2 and 3 are almost the same but featuring 256 and 128 neurons correspondingly. The fourth layer is using 64 neurons and TANH as the activation function. Finally, the fifth layer is the output layer. 
<br> As the loss function, we will be using the Mean Absolute Error (MAE). As the optimizer, we are using Adam initialized with a value of 0.001. </font>

In [ ]:
def build_compile_model():
    model = keras.Sequential([
        normalizer,
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='tanh'),
        layers.Dense(1)
    ])

    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.001))
    
    return model

model = build_compile_model()

<font size=3><br> Below is a summary of the Neural Network model. </font>

In [ ]:
model.summary()

<font size=3><br> We are now going to fit the model for 100 epochs and using a 20% validation split. </font>

In [ ]:
num_epochs = 100

history = model.fit(
    X_train_df, 
    y_train,
    epochs=num_epochs, 
    validation_split=0.2,
    verbose=1)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

<font size=3><br> Here is a plot of the history. The evolution of the loss and val_loss during the epochs can be seen. </font>

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.legend()
    plt.grid(True)
    
plot_loss(history)

In [ ]:
model.evaluate(X_test, y_test, verbose=0)

<font size=3><br> Now let's build the model again, but this time early stopping is going to be used, in order to prevent the model from overfittinng. </font>

In [ ]:
model = build_compile_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(X_train_df, y_train, epochs=num_epochs,
                    validation_split = 0.2, verbose=1, 
                    callbacks=[early_stop])

<font size=3><br> Finally, let's use the model on the test data and check its predictions. We are going to plot the predictions in the same diagram with the true valence in order to see the model's performance. </font>

In [ ]:
test_predictions = model.predict(X_test).flatten()

a = plt.axes(aspect='equal')
plt.scatter(y_test, test_predictions)
plt.xlabel('True Valence')
plt.ylabel('Prediction')
lims = [0, 1]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

<font size=3><br> So overall, the model's performance is good. There is an RMSE of around 0.15 but as it can be seen on the plot above the predicted values are close to the y=x line  </font>